<a href="https://colab.research.google.com/github/priya-dagar/NLP--PIPELINES/blob/main/NLP_PROJECT_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune a Pre-Trained LLM for Text Classification

Project: Fine-Tune a Pre-Trained LLM Using a Custom Dataset
Dataset: AG News (topic classification, 4 classes)
Model: distilbert-base-uncased

Sections:
1. Setup & Imports
2. Load Dataset
3. Preprocessing & Train/Val/Test Split
4. Tokenization
5. Model, Trainer, TrainingArguments
6. Evaluation & Metrics (Accuracy, Precision, Recall, F1)
7. Confusion Matrix (optional)
8. Save Model
9. Optional: Streamlit app for inference
10. Short report (2-3 pages) in markdown
11. screenshots of output of main codes

Notes: to run as notebook, split code cells accordingly. This script uses `datasets`, `transformers`, and `sklearn`.


In [1]:
# Cell 1: Install and Import Libraries
!pip install --upgrade datasets transformers wandb scikit-learn matplotlib


import numpy as np
import re
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import wandb
import torch

In [2]:
# Load Dataset
# Use AG News dataset
dataset = load_dataset("ag_news")

train_ds = dataset["train"]
test_ds = dataset["test"]

# FAST TRAINING: use a small subset
train_small = train_ds.select(range(2000))   # 2k samples
test_small  = test_ds.select(range(500))     # 500 samples



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [3]:
# Preprocessing & Split
full_train = dataset['train'].shuffle(seed=42)
train_testval = full_train.train_test_split(test_size=0.20, seed=42)  # 80/20
train_ds = train_testval['train']
val_ds = train_testval['test']
test_ds = dataset['test']

# Optional cleaning
import re

def clean_text(example):
    text = example['text'].strip().lower()
    text = re.sub(r"\s+", " ", text)
    example['text'] = text
    return example

train_ds = train_ds.map(clean_text)
val_ds = val_ds.map(clean_text)
test_ds = test_ds.map(clean_text)

Map:   0%|          | 0/96000 [00:00<?, ? examples/s]

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")   # <--- ADD THIS

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_tok = train_small.map(tokenize, batched=True)
test_tok  = test_small.map(tokenize, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [5]:
# Model and Metrics
model_name = "distilbert-base-uncased"
num_labels = 4
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds, average='macro'),
        'recall': recall_score(labels, preds, average='macro'),
        'f1': f1_score(labels, preds, average='macro')
    }

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Training Arguments (Old HF compatible)
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    num_train_epochs=1,      # 1 epoch only
    max_steps=400,           # limit training to 400 steps
    logging_steps=20,
    eval_strategy="epoch",
    save_strategy="no",
    report_to="wandb"        # enable W&B
)


In [7]:
# Trainer
trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_tok,
eval_dataset=test_tok,
tokenizer=tokenizer,
compute_metrics=compute_metrics
)

/tmp/ipython-input-82592342.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train
trainer.train()
# Evaluate on Test Set
metrics = trainer.evaluate(eval_dataset=test_tok)
print("Test metrics:", metrics)
preds_output = trainer.predict(test_tok)
logits = preds_output.predictions
preds = np.argmax(logits, axis=-1)
labels = preds_output.label_ids
report = classification_report(labels, preds, target_names=["World", "Sports", "Business", "Sci/Tech"], digits=4)
print(report)
cm = confusion_matrix(labels, preds)
print("Confusion matrix:\n", cm)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 2301730257 (2301730257-krmangalam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


In [ ]:
# Plot Confusion Matrix
fig, ax = plt.subplots(figsize=(6,6))
ax.matshow(cm, cmap=plt.cm.Blues)
# Correct indentation here
for (i, j), val in np.ndenumerate(cm):
    ax.text(j, i, int(val), ha='center', va='center')

ax.set_xlabel('Predicted')
ax.set_ylabel('True')
ax.set_xticks(range(num_labels))
ax.set_yticks(range(num_labels))
ax.set_xticklabels(["World", "Sports", "Business", "Sci/Tech"], rotation=45)
ax.set_yticklabels(["World", "Sports", "Business", "Sci/Tech"])

plt.tight_layout()
plt.savefig('confusion_matrix.png')
plt.show()

In [ ]:
# Save Model & Tokenizer
model.save_pretrained('./fine_tuned_distilbert_agnews')
tokenizer.save_pretrained('./fine_tuned_distilbert_agnews')